#Proyecto Final - Parte Machine Learning - STEM women

Este proyecto busca desarrollar un sistema de recomendación de iniciativas STEM personalizadas según el tipo de actividad, el formato y la edad del usuario, así como un modelo de predicción.

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import lightgbm as lgb
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE


In [ ]:
df_iniciativas = pd.read_excel('/content/df_2019_2024_limpio.xlsx')


In [ ]:
df_iniciativas.columns = df_iniciativas.columns.str.strip().str.lower()

In [ ]:
df_iniciativas.columns.tolist()

# SELECCIÓN E INGENIERÍA DE CARACTERÍSTICAS

In [ ]:
# Selecionar colunas numéricas para análise
df_num = df_iniciativas.select_dtypes(include=['int64', 'float64'])

# Verificar se há valores faltantes
print(df_num.isnull().sum())

# Visualizar correlação com Total_impactos
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 8))
sns.heatmap(df_num.corr()[['total_impactos']].sort_values(by='total_impactos', ascending=False), annot=True, cmap='viridis')
plt.title("Correlación com Total_impactos")
plt.show()

In [ ]:
# Seleccionar columnas numéricas para el análisis
df_num = df_iniciativas.select_dtypes(include=['int64', 'float64'])

# Generar la matriz de correlación
matriz_correlacion = df_num.corr()

# Mostrar la matriz de correlación
print(matriz_correlacion)

✅** Importante: Este resultado no se generará por motivos de protección de datos.**

📊 Correlation Analysis

We generated a heatmap to visualize the correlation between numerical variables and total_impactos.

    The variables with the highest positive correlation with total impact were:

        13_14_años → 0.74

        Total_Impactos_Carrera → 0.72

        inicio_GradoFP → 0.68

    Variables with low correlation included:

        evento, blog, recursos, and some specific regions.

🧠 Conclusions

    Age group and career stage are strongly related to the success of the initiatives (higher impact).

    Variables like type of content (events, blogs) and location have less direct influence.

    These findings help guide the personalization of future recommendations.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

X = df_iniciativas.select_dtypes(include=['int64', 'float64']).drop(columns=['total_impactos'])
y = df_iniciativas['total_impactos']

model = RandomForestRegressor()
model.fit(X, y)

importancia = pd.Series(model.feature_importances_, index=X.columns).sort_values(ascending=False)
print(importancia.head(10))

 Modelo de Random Forest

    Entrenamos un modelo de RandomForestRegressor para identificar la importancia real de las variables en la predicción del impacto.

    Algunas variables con baja correlación, como formación, mentoría y evento, mostraron alta importancia para el modelo.

    Esto se debe a que el modelo considera interacciones no lineales entre variables.

 ## Recomendador Personalizado por Perfil:

 Este algoritmo recomenda iniciativas com base no perfil do usuário, aplicando filtros e medindo a similaridade com iniciativas existentes.
🧾 Entradas do Perfil

    edad: Faixa etária ou etapa da carreira (ex: final_GradoMaster)

    formato: Preferência de formato (Presencial, Online, Híbrida)

    actividad_preferida: Tipo de atividade desejada (ex: formacion, mentoría)

⚙️ Processo da Recomendação

    Padronização de colunas

        Corrige nomes e formata strings para minúsculas.

    Mapeamento de importância

        Converte categorias (como formacion, evento) em valores numéricos via mapeo_importancia.

    Filtros aplicados:

        🔹 Idade/etapa: Filtra iniciativas compatíveis com a faixa etária informada.

        🔹 Formato: Mantém apenas as iniciativas no formato preferido.

        🔹 Atividade preferida: Exige um nível mínimo de importância (≥ 2) para a atividade desejada.

    Criação de vetores de atividade

        Gera vetores para iniciativas e para o usuário.

    Normalização (MinMaxScaler)

        Normaliza os dados para que fiquem no mesmo intervalo (0 a 1).

    Cálculo de similaridade (Cosine Similarity)

        Compara o vetor do usuário com os vetores das iniciativas.

        Retorna as iniciativas mais semelhantes.



In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity

# ======== CONFIGURACIONES ========
actividad_preferida = [
    'formacion', 'evento', 'mentoring', 'blog', 'rrss',
    'bolsa_trabajo', 'networking', 'recursos', 'emprendimiento'
]

mapeo_importancia = {
    'no se realiza': 0,
    'poco importante': 1,
    'algo importante': 2,
    'importante': 3,
    'muy importante': 4
}

columnas_edad = [
    '1_5_años', '6_9_años', '10_12_años', '13_14_años', '15_16_años',
    'bachillerato', 'inicio_gradofp', 'final_gradomaster',
    'etapa_junior', 'etapa_consolidacion', 'etapa_madurez'
]

formatos_validos = ['online', 'presencial', 'híbrida']

# ======== FUNCIÓN PRINCIPAL ========
def recomendar_por_perfil(perfil, df, actividades, top_n=5):
    df = df.copy()
    df.columns = df.columns.str.strip().str.lower()  # normaliza nombres de columnas

    # Mapeo de valores de importancia a números
    for col in actividades:
        if col in df.columns:
            df[col] = df[col].fillna('no se realiza').astype(str).str.strip().str.lower().map(mapeo_importancia).fillna(0)

    col_edad = perfil['edad'].strip().lower()
    formato_usuario = perfil['formato'].strip().lower()
    act = perfil['actividad_preferida'].strip().lower()

    # Filtro por edad
    if col_edad in columnas_edad and col_edad in df.columns:
        filtro = df[df[col_edad] > 0].copy()
    else:
        filtro = df.copy()
    print("➡️ Total después del filtro por edad:", filtro.shape[0])

    # Filtro por formato
    if formato_usuario in formatos_validos and formato_usuario in df.columns:
        filtro = filtro[filtro[formato_usuario] == 1].copy()
    print("➡️ Total después del filtro por formato:", filtro.shape[0])

    # Filtro por actividad preferida (nivel mínimo = 2)
    if act in actividades:
        filtro = filtro[filtro[act] >= 2].copy()
    print("➡️ Total después del filtro por actividad preferida:", filtro.shape[0])

    if filtro.empty:
        print("⚠️ No se encontraron iniciativas después de aplicar los filtros.")
        return pd.DataFrame()

    # Vectores de iniciativas (actividades)
    sub = filtro[actividades].copy()

    # Normalización con MinMaxScaler
    scaler = MinMaxScaler()
    sub_norm = scaler.fit_transform(sub)

    # Vector del usuario
    vector = [0] * len(actividades)
    if act in actividades:
        vector[actividades.index(act)] = 3

    # Normalización del vector del usuario
    vector_norm = scaler.transform([vector])

    # Cálculo de similitud
    similitudes = cosine_similarity(vector_norm, sub_norm)[0]
    filtro = filtro.copy()
    filtro['similitud'] = similitudes

    return filtro.sort_values(by='similitud', ascending=False).head(top_n)[
        ['nombre_iniciativa', 'formato_simplificado', 'similitud']
    ]

# ======== EJEMPLO DE USO ========
# Suponiendo que df_iniciativas ya ha sido cargado (ej. con pd.read_csv)
# y contiene columnas como 'formacion', 'evento', etc.

perfil_usuario = {
    'edad': 'final_GradoMaster',
    'formato': 'Presencial',
    'actividad_preferida': 'formacion'
}

resultados = recomendar_por_perfil(perfil_usuario, df_iniciativas, actividad_preferida)

if not resultados.empty:
    print(resultados)
else:
    print("No se encontraron iniciativas para este perfil.")

✅** Importante: Este resultado no se generará por motivos de protección de datos.**

## Evaluación del Modelo

In [ ]:
from sklearn.metrics import precision_score, recall_score

# Simular um usuário fictício
perfil_usuario_ficticio = {
    'edad': 'final_GradoMaster',   # Faixa etária preferida
    'formato': 'presencial',       # Formato preferido
    'actividad_preferida': 'formacion'  # Atividade preferida
}

# Gerar as recomendações para esse usuário fictício
recomendaciones = recomendar_por_perfil(perfil_usuario_ficticio, df_iniciativas, actividad_preferida)

# Agora simular a relevância de cada recomendação
relevancia_predita = []
y_true = []

# Simular manualmente se a recomendação seria relevante
for _, row in recomendaciones.iterrows():
    # Verificar apenas atividade e formato (não precisa da idade para relevância)
    atividade_relevante = perfil_usuario_ficticio['actividad_preferida'].lower() in row['nombre_iniciativa'].lower()
    formato_relevante = perfil_usuario_ficticio['formato'].lower() in row['formato_simplificado'].lower()

    # Relevante se a atividade e formato coincidirem
    is_relevant = 1 if (atividade_relevante and formato_relevante) else 0
    relevancia_predita.append(is_relevant)
    y_true.append(is_relevant)  # Para simular, marcamos como relevante quando é relevante

# Calcular Precision e Recall
precision = precision_score(y_true, relevancia_predita, zero_division=0)
recall = recall_score(y_true, relevancia_predita, zero_division=0)

# Exibir resultados
print(f"Precision: {precision}")
print(f"Recall: {recall}")


**Posibles causas del bajo rendimiento del modelo:**

No hay datos reales de usuarios (ground truth).

y_true y relevancia_predita fueron definidos con las mismas reglas del modelo → genera circularidad


**Resultado: **
Métricas como Precision = 0.0, Recall = 0.0 no reflejan la calidad real del sistema.


**Evaluación inválida porque:**
El modelo no está siendo probado contra elecciones reales.
Solo confirma lo que él mismo ya filtró.
Las métricas supervisadas solo tienen sentido con retroalimentación del usuario.


##**Otra alternativa de métricas de evaluación**

In [ ]:
# ======== EVALUACIÓN ========
def evaluacion_modelo(df, recomendaciones, perfil_usuario, actividades):
    recomendados = recomendaciones['nombre_iniciativa'].values.tolist()

    # Simulación de ground truth: iniciativas con alto valor para la actividad preferida
    act = perfil_usuario['actividad_preferida'].strip().lower()
    actividades_col = [a.lower() for a in actividades]

    if act not in actividades_col:
        print(f"⚠️ La actividad '{act}' no está entre las actividades válidas.")
        return 0, 0, 0

    reales = df[df[act] >= 3]['nombre_iniciativa'].values.tolist()
    acertadas = len(set(recomendados).intersection(reales))
    precision = acertadas / len(recomendados) if recomendados else 0

    # Simulación de RMSE
    vector_perfil = [0] * len(actividades_col)
    vector_perfil[actividades_col.index(act)] = 3

    df_activ = df[actividades_col].fillna(0)
    similitudes = cosine_similarity([vector_perfil], df_activ.values)[0]
    predicciones = similitudes[:len(recomendados)]
    rmse = np.sqrt(mean_squared_error([3]*len(predicciones), predicciones)) if predicciones.any() else 0

    # Simulación de tasa de engagement
    engagement_simulado = np.random.rand(len(recomendados))
    tasa_engagement = np.mean(engagement_simulado)

    print(f"➡️ Precisión simulada: {precision:.2f}")
    print(f"➡️ RMSE simulado: {rmse:.2f}")
    print(f"➡️ Engagement simulado: {tasa_engagement:.2f}")

    return precision, rmse, tasa_engagement

# ======== EJEMPLO DE USO ========
# Asegúrate de que tu DataFrame esté cargado (por ejemplo: df_iniciativas = pd.read_csv("..."))
# Y que los nombres de columnas coincidan con los esperados.

perfil_usuario = {
    'edad': 'final_GradoMaster',
    'formato': 'Presencial',
    'actividad_preferida': 'formacion'  # elige una columna que exista
}

# Ejecutar
try:
    recomendaciones = recomendar_por_perfil(perfil_usuario, df_iniciativas, actividad_preferida)
    if not recomendaciones.empty:
        print(recomendaciones)
        evaluacion_modelo(df_iniciativas, recomendaciones, perfil_usuario, actividad_preferida)
    else:
        print("⚠️ No se encontraron recomendaciones.")
except Exception as e:
    print("Error:", e)
    print("Columnas disponibles en el dataset:")
    print(df_iniciativas.columns.tolist())

 ✅ "Importante: Este resultado no se generará por motivos de protección de datos."


Dado que no tenemos datos reales de usuario, utilizamos una evaluación simulada:
Precisión simulada: Compara las recomendaciones con las iniciativas relevantes según el perfil del usuario.

RMSE simulado: Mide la diferencia entre las preferencias del usuario y las recomendaciones.

Engagement simulado: Genera valores aleatorios para simular la tasa de interacción.

Evaluación Cualitativa:
Verifica manualmente si las recomendaciones tienen sentido.
Pide la evaluación de expertos.

Limitaciones:
Las métricas no reflejan interacciones reales, solo aproximaciones del modelo.
Próximos pasos: Evaluar con datos reales para usar métricas como Precision@N, Recall@N y NDCG.

# Modelo de Predición

Algoritmo
Regresión Lineal (LinearRegression de scikit-learn)
Relación lineal entre impacto de iniciativas y matrículas

Objetivo: Predicción de Matrículas a partir del Impacto de las Iniciativas
Unión de DataFrames:
Combinamos df_iniciativas (impacto de iniciativas) con df_matriculadas (matrículas en bachillerato) usando la columna año.
Modelo de Predicción:
Utilizamos regresión lineal para predecir las matrículas en los próximos 5 años basándonos en el impacto de las iniciativas pasadas.
Resultados:
Predicción de matrículas para los próximos años y visualización en un gráfico.
La matriz de correlación mostró la relación entre el impacto de las iniciativas y las matrículas en bachillerato.

In [ ]:
df_matriculadas = pd.read_excel('/content/Mujeres matriculadas em Ciências y Tecnologia -Grado_Ciclo_Bachillerato - 2011-2024 Ciencias(1).xlsx')
df_matriculadas

In [ ]:
df_matriculadas.columns = df_matriculadas.columns.str.strip()  # Remove espaços e quebras de linha no início/fim

In [ ]:
df_matriculadas.columns = df_matriculadas.columns.str.replace(r'\s+', ' ', regex=True).str.strip()

In [ ]:
df_matriculadas

In [ ]:
# 1. Renombrar columnas para asegurar compatibilidad
df_matriculadas.rename(columns={'Año': 'año', 'Bachillerato': 'matriculas_bachillerato'}, inplace=True)
df_iniciativas.rename(columns={'Año': 'año', 'total_impactos': 'impacto_iniciativas'}, inplace=True)

# 2. Realizar el merge (unión) de los dos DataFrames por el año
df_combinado = pd.merge(df_iniciativas, df_matriculadas, on='año', how='inner')



# 3. Eliminar filas con valores NaN en las columnas de interés
variables = ['impacto_iniciativas', 'matriculas_bachillerato', 'Grado y Ciclo']
df_combinado = df_combinado.dropna(subset=variables)

# 4. Calcular y mostrar la matriz de correlación como un heatmap
import seaborn as sns
import matplotlib.pyplot as plt

matriz_correlacion = df_combinado[variables].corr()

plt.figure(figsize=(8, 6))
sns.heatmap(matriz_correlacion, annot=True, cmap='coolwarm', fmt='.2f', vmin=-1, vmax=1)
plt.title("Matriz de Correlación de las Variables Seleccionadas")
plt.show()


**Qué muestra el resultado?**

impacto_iniciativas y matriculas_bachillerato tienen una correlación muy alta (0.88), lo que indica que cuando aumenta una, la otra también tiende a aumentar.
matriculas_bachillerato y Grado y Ciclo también tienen una correlación positiva considerable (0.67).
impacto_iniciativas y Grado y Ciclo tienen una correlación positiva moderada (0.49).
Esto sugiere que existe una relación importante entre el impacto de las iniciativas y el número de matrículas, y que también hay relación con el grado/ciclo educativo.

**Evaluación del Modelo**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

# Suponha que os DataFrames df_matriculadas e df_iniciativas já foram carregados e renomeados
# Como você já fez:
# df_matriculadas.rename(columns={'Año': 'año', 'Bachillerato': 'matriculas_bachillerato'}, inplace=True)
# df_iniciativas.rename(columns={'Año': 'año', 'total_impactos': 'impacto_iniciativas'}, inplace=True)

# 1. Merge
df_combinado = pd.merge(df_iniciativas, df_matriculadas, on='año', how='inner')

# 2. Eliminar NaNs
variables = ['impacto_iniciativas', 'matriculas_bachillerato', 'Grado y Ciclo']
df_combinado = df_combinado.dropna(subset=variables)

# 3. Treinar modelo de regressão linear
X = df_combinado[['impacto_iniciativas']]
y = df_combinado['matriculas_bachillerato']

modelo = LinearRegression()
modelo.fit(X, y)

# 4. Prever matrículas para os próximos 5 anos
ultimo_ano = df_combinado['año'].max()
anos_futuros = [ultimo_ano + i for i in range(1, 6)]

# Simular crescimento dos impactos futuros (ex: crescimento linear)
ultimo_impacto = df_combinado['impacto_iniciativas'].iloc[-1]
incremento = 3000  # Ajuste conforme necessário
impactos_futuros = np.array([[ultimo_impacto + incremento * i] for i in range(1, 6)])

prediccion_matriculas = modelo.predict(impactos_futuros)

# 5. Plotar tudo
plt.figure(figsize=(10, 6))

# Matrículas históricas
plt.plot(df_combinado['año'], df_combinado['matriculas_bachillerato'], marker='o', label='Matrículas Históricas')

# Impacto das iniciativas (no mesmo eixo para visualização, mas são escalas diferentes!)
plt.plot(df_combinado['año'], df_combinado['impacto_iniciativas'], marker='x', label='Impacto de las Iniciativas', color='orange')

# Predição de matrículas
plt.plot(anos_futuros, prediccion_matriculas, linestyle='--', marker='s', color='green', label='Predicción Matrículas')

# Personalização
plt.xlabel("Año")
plt.ylabel("Número de Matrículas")
plt.title("Predicción de Matrículas en Áreas Científicas Basada en el Impacto de las Iniciativas")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

¿Qué muestra el gráfico?
Matrículas históricas: crecimiento leve y estable hasta 2022.
Impacto de iniciativas: gran variabilidad, fuerte aumento en 2023.
Predicción 2024–2028: matrículas se mantienen constantes (~153.398).
Interpretación
El modelo predice estabilidad, sin reflejar el aumento del impacto.
Indica que el modelo es conservador o insensible a cambios recientes.
Podría haber factores no incluidos que influyen más que el impacto.
 Limitaciones del análisis
Falta de datos previos a las iniciativas.
Escasez de datos desagregados por género (mujeres).
Esto limita la evaluación real del impacto de las iniciativas.

#Evaluación del Modelo

R² (Coeficiente de Determinación)
Valor entre 0 e 1 (quanto mais perto de 1, melhor).
Mede a proporção da variância explicada pelo modelo.
 Ideal para ver o ajuste geral.
MAE (Error Absoluto Medio)
Média das diferenças absolutas entre previsões e valores reais.
Fácil de interpretar (mesma unidade que a variável prevista).
MSE (Error Cuadrático Medio)
Erros elevados são penalizados mais fortemente.
Sensível a outliers.
RMSE (Raíz del MSE)
Interpretação mais intuitiva do MSE (mesma unidade das matrículas).

In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import numpy as np

y_pred = modelo.predict(X)
print("R²:", r2_score(y, y_pred))
print("MAE:", mean_absolute_error(y, y_pred))
print("RMSE:", np.sqrt(mean_squared_error(y, y_pred)))

AFINACIÓN DE HIPERPARÁMETROS

Técnica utilizada: GridSearchCV
Validación cruzada: LeaveOneOut (ideal para pocos datos)

Modelo afinado: Ridge (Regresión con regularización)
Hiperparámetro ajustado: alpha
Valores probados: 0.01, 0.1, 1, 10, 100
Mejor resultado: alpha = 0.01

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge

param_grid = {'alpha': [0.01, 0.1, 1, 10, 100]}
grid = GridSearchCV(Ridge(), param_grid, cv=5, scoring='r2')
grid.fit(X, y)

print("Melhor parâmetro:", grid.best_params_)
print("Melhor R²:", grid.best_score_)

Afinação de hiperparametros do modelo de predicion

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV, LeaveOneOut
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import numpy as np

# Defina seus dados (X e y já devem estar prontos)
# Exemplo: X = df_combinado[['impacto_iniciativas']], y = df_combinado['matriculas_bachillerato']

# Configuração do Leave-One-Out
loo = LeaveOneOut()

# Grade de hiperparâmetros para testar
param_grid = {'alpha': [0.01, 0.1, 1, 10, 100]}

# Criação do GridSearch com validação Leave-One-Out
grid = GridSearchCV(Ridge(), param_grid, cv=loo, scoring='r2')
grid.fit(X, y)

# Melhor modelo encontrado
mejor_modelo = grid.best_estimator_

# Avaliação do modelo final (previsão no mesmo conjunto — ou você pode usar divisão treino/teste)
y_pred = mejor_modelo.predict(X)

# Métricas
r2 = r2_score(y, y_pred)
mae = mean_absolute_error(y, y_pred)
rmse = np.sqrt(mean_squared_error(y, y_pred))

# Resultados
print("Mejor parámetro (alpha):", grid.best_params_)
print(f"R²: {r2:.4f}")
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")

5. CONCLUSIONES

El modelo de predicción muestra estabilidad en las matrículas para 2024–2028, sin reflejar el reciente aumento por impacto de iniciativas, lo que indica un enfoque conservador y posibles limitaciones por falta de datos históricos y desagregados.

El modelo de recomendación, evaluado de forma simulada ante la ausencia de datos reales de usuario, ofrece resultados prometedores, aunque las métricas actuales solo aproximan el rendimiento real.

Ambos modelos enfrentan desafíos por la escasez y calidad de los datos, lo que limita la evaluación precisa del impacto y la personalización.

Como próximos pasos, es clave incorporar datos más completos y reales para validar y mejorar la capacidad predictiva y la relevancia de las recomendaciones, utilizando métricas especializadas como Precision@N, Recall@N y NDCG.

Somos conscientes de las carencias que tiene la base de datos: principalmente, la alta disparidad en los valores de impacto entre iniciativas, muchas con bajo alcance y unas pocas con impactos muy elevados pero que distorsionan la media. Por ello, como mejora futura que ya se está trabajando, se plantea la creación de un indicador de impacto ponderado, que tenga en cuenta factores como la duración de la iniciativa, la frecuencia con la que se repite y el tipo de actividad. Esto permitiría obtener una visión más justa y representativa del impacto real. Sin embargo, resulta muy complejo, ya que cada iniciativa suele englobar múltiples actividades con características distintas, dificultando una respuesta clara sobre duración y repetición.